# STag II Demonstration

The following iPython Jupyter notebook gives a step-by-step demonstration of how to use STag to get the tag probabilities, equivalent widths, and the predicted class for a spectra.

# Setup

The first step is to read in the beta values for each of the tags as well as an example spectrum (this can be modified to read in an appropriate spectrum of your choice).

In [1]:
import beta_readerII
import numpy as np
import os 

path = os.getcwd()
beta = beta_readerII.beta_reader(path)
spectra = '%s/tns_2021qtc_2021-06-25_08-40-51.161193_P60_SEDM_ZTF.ascii' % path
name = ['SN 2021qtc']
z = [0.081]

# Pre-processing

In order to use STag, spectra need to be pre-processed appropriately. This involves filtering, de-redshifting, binning, continuum removal, apodisation.

All of these steps are handled by the spectra_preprocessing package, which largely uses methods made for the software [DASH](https://github.com/daniel-muthukrishna/astrodash).

In [2]:
import spectra_preprocessing as sp
from astropy.io import fits

#Read in the data file of the spectra and extract the flux and wavelength
data = np.genfromtxt(spectra)
wave = data[:,0]
flux = data[:,1]

full = np.column_stack((wave, flux))

#Initialise for pre-processing
preProcess = sp.PreProcessing(full, 2500, 10000, 1500)

#Do the pre-processing steps
sfWave, sfFlux, minInd, maxInd, sfZ, sfArea = preProcess.two_column_data(z[0], smooth=6, minWave=2500, maxWave=10000)

# Cutting the Spectra

Many of the tags use specific wavelength ranges of the spectrum rather than the whole thing and so we create multiple instances of the original spectrum cut at the corresponding wavelengths for each tag. 

In [3]:
import EW_calculator

class feature_data(object):
    """a class for holding the wavelength and flux for a specific tag."""
    def __init__(self, label):
        self.label = label
        self.wavelength = []
        self.flux = []
        self.eqw = []
        
cuts = np.genfromtxt('%s/cuts_v2.txt' % path, dtype=int)

#Silicon 6355
si_6355_tag = feature_data('Si 6355')
si_6355_tag.wavelength = cuts[0]
si_6355_tag.flux = sfFlux[si_6355_tag.wavelength[0]:si_6355_tag.wavelength[1]]
si_6355_tag.eqw = EW_calculator.eqw(si_6355_tag.wavelength[0],si_6355_tag.wavelength[1],sfWave,sfFlux)

#Calcium H&K
ca_tag = feature_data('Ca H&K')
ca_tag.wavelength = cuts[1]
ca_tag.flux = sfFlux[ca_tag.wavelength[0]:ca_tag.wavelength[1]]
ca_tag.eqw = EW_calculator.eqw(ca_tag.wavelength[0],ca_tag.wavelength[1],sfWave,sfFlux)

#Sulphur
s_tag = feature_data('S')
s_tag.wavelength = cuts[2]
s_tag.flux = sfFlux[s_tag.wavelength[0]:s_tag.wavelength[1]]
s_tag.eqw = EW_calculator.eqw(s_tag.wavelength[0],s_tag.wavelength[1],sfWave,sfFlux)

#Hydrogen Alpha Narrow
ha_tag = feature_data('H Alpha Narrow')
ha_tag.wavelength = cuts[3]
ha_tag.flux = sfFlux[ha_tag.wavelength[0]:ha_tag.wavelength[1]]
ha_tag.eqw = EW_calculator.eqw(ha_tag.wavelength[0],ha_tag.wavelength[1],sfWave,sfFlux)

#Helium 5876
he_tag = feature_data('He 5876')
he_tag.wavelength = cuts[4]
he_tag.flux = sfFlux[he_tag.wavelength[0]:he_tag.wavelength[1]]
he_tag.eqw = EW_calculator.eqw(he_tag.wavelength[0],he_tag.wavelength[1],sfWave,sfFlux)

#Iron 5170
fe_tag = feature_data('Fe 5170')
fe_tag.wavelength = cuts[5]
fe_tag.flux = sfFlux[fe_tag.wavelength[0]:fe_tag.wavelength[1]]
fe_tag.eqw = EW_calculator.eqw(fe_tag.wavelength[0],fe_tag.wavelength[1],sfWave,sfFlux)

#Silicon 4000
si_4000_tag = feature_data('Si 4000')
si_4000_tag.wavelength = cuts[6]
si_4000_tag.flux = sfFlux[si_4000_tag.wavelength[0]:si_4000_tag.wavelength[1]]
si_4000_tag.eqw = EW_calculator.eqw(si_4000_tag.wavelength[0],si_4000_tag.wavelength[1],sfWave,sfFlux)

#Hydrogen Beta
hb_tag = feature_data('H Beta')
hb_tag.wavelength = cuts[7]
hb_tag.flux = sfFlux[hb_tag.wavelength[0]:hb_tag.wavelength[1]]
hb_tag.eqw = EW_calculator.eqw(hb_tag.wavelength[0],hb_tag.wavelength[1],sfWave,sfFlux)

#Hydrogen Alpha Wide
haw_tag = feature_data('H Alpha Wide')
haw_tag.wavelength = cuts[8]
haw_tag.flux = sfFlux[haw_tag.wavelength[0]:haw_tag.wavelength[1]]
haw_tag.eqw = EW_calculator.eqw(haw_tag.wavelength[0],haw_tag.wavelength[1],sfWave,sfFlux)

# Tagging

With spectra pre-processed and the necessary cuts made, we can now get the tag probabilities of the spectra and add them to an array ready to be given to the trained classifier.

In [4]:
from tagging import log_reg_two

final = np.zeros([1,18])

#Assign tag probablities
#H alpha narrow
ha_result = log_reg_two(ha_tag.flux, beta[4])
final[:,0] = ha_result

#H beta
hb_result = log_reg_two(hb_tag.flux, beta[5])
final[:,1] = hb_result

#Si 4000
si_4000_result = log_reg_two(si_4000_tag.flux, beta[0])
final[:,2] = si_4000_result

#Si 6355
si_6355_result = log_reg_two(si_6355_tag.flux, beta[2])
final[:,3] = si_6355_result

#S W
s_result = log_reg_two(s_tag.flux, beta[1])
final[:,4] = s_result

#He 5876
he_result = log_reg_two(he_tag.flux, beta[3])
final[:,5] = he_result

#Ca H&K
ca_result = log_reg_two(ca_tag.flux, beta[6])
final[:,6] = ca_result

#Fe 5170
fe_result = log_reg_two(fe_tag.flux, beta[7])
final[:,7] = fe_result

#He Eq. Width
final[:,8] = he_tag.eqw

#H alpha Eq. Width
final[:,9] = ha_tag.eqw

#H beta Eq. Width
final[:,10] = hb_tag.eqw

#Ca HK Eq. Width
final[:,11] = ca_tag.eqw

#Si 4000 Eq. Width
final[:,12] = si_4000_tag.eqw

#Si 4000 prob. * eqw
final[:,13]  = si_4000_result * si_4000_tag.eqw

#Fe 5170 Eq. Width
final[:,14] = fe_tag.eqw

#Si 6150 Eq. Width
final[:,15] = si_6355_tag.eqw

#H alpha wide
haw_result = log_reg_two(haw_tag.flux, beta[8])
final[:,16] = haw_result

#H alpha wide Eq. Width
final[:,17] = haw_tag.eqw

# Classifying

We can now make our predictions for the class of the supernova by using the trained model. Since we are using softmax, we use 'np.argmax' to select the class with the highest probability, though one can see the probabilities of all the classes by printing 'class_prob'.

The predicted class is given a number, which corresponds to one of the 5 possible classes:

0 = Type Ia

1 = Type Ib

2 = Type Ic

3 = Type II

In [ ]:
import keras
v = keras.__version__
from packaging import version
if version.parse(v) < version.parse('2.5.0'):
    print("You may need to update Keras")

#Load in the trained model
model = keras.models.load_model('%s/STagV4_model_v1' % path)

#Make classification prediction
class_prob = model.predict(final)
preds = np.argmax(class_prob, axis=-1)

sfFlux = [sfFlux]
flux = [flux]
wave = [wave]

path2 = '%s/CC Models' % path
classification = z_checking.z_check(path,path2,name,z,flux,wave,sfFlux,sfWave,preds,beta[0],beta[1],beta[2],beta[3],beta[4],beta[5],beta[6],beta[7],beta[8],[final[0][0]],[final[0][1]],[final[0][2]],[final[0][3]],[final[0][4]],[final[0][5]],[final[0][6]],[final[0][7]],[final[0][16]],[final[0][8]],[final[0][9]],[final[0][10]],[final[0][11]],[final[0][12]],[final[0][14]],[final[0][15]],[final[0][17]])